# 산업융합형 인공지능 청년혁신가 양성과정 팀 프로젝트(딥링크)

# Content Based

In [ ]:
import pandas as pd  
import numpy as np  

In [ ]:
things_df = pd.read_excel('특산물.xlsx') # 데이터 불러오기

In [ ]:
# 데이터에 속한 label 벡터화
from sklearn.feature_extraction.text import CountVectorizer
count_vect2 = CountVectorizer(min_df=1, ngram_range=(1, 1))  
data_mat2 = count_vect2.fit_transform(things_df['label'])
print(data_mat2.shape)
print(data_mat2)

In [ ]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
data_sim = cosine_similarity(data_mat2, data_mat2)

In [ ]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 특산물을 앞에서부터 순서대로 보여줌
data_sim_sorted_ind = data_sim.argsort()[:, ::-1] # ::-1 : 역순으로 정렬

In [ ]:
# 단순 유사도로 특산물 추천
# 예시 상추
# 특산물 이름이 상추와 같은 상품 추출
name_things = things_df[things_df['name'] == '상추']
name_things

In [ ]:
# 특산물 인덱스를 반환하고 특산물은은 다른 지역의 같은 이름을 가진 상품들이 존재하기 때문에 여러개의 값 출력
name_index = name_things.index.values
name_index   

In [ ]:
# 해당 특산물과 유사한 특산물 인덱스 추출 
similar_indexes = data_sim_sorted_ind[name_index, :30]
similar_indexes

In [ ]:
# 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
# dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
similar_indexes = similar_indexes.reshape(-1)

In [ ]:
# 새로운 데이터 프레임에 저장
things_df2 = things_df.iloc[similar_indexes]

In [ ]:
# 지역은 다르고 이름이 같기에 서로 중복되는 데이터 존재
a = things_df2[things_df2['name']=='상추']
a

In [ ]:
# 중복 제거
# 같은 품목 다른 지역 리스트 제공
a = a.drop_duplicates()
a

In [ ]:
# 상추와 유사한 특산물 뽑아내기
# 상추를 제외한 특산물
b = things_df2[things_df2['name']!='상추']
b

In [ ]:
# 마찬가지로 중복제거
# 상추를 제외한 특산물 중 유사한 순으로 제공
b = b.drop_duplicates()
b

# CF

In [ ]:
import pandas as pd
import numpy as np

thing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/특산물.xlsx')
ratings = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/특산물 클릭.xlsx')

print(thing.shape)
print(ratings.shape)

In [ ]:
# pivot_table 메소드를 사용해서 사용자-클릭수 행렬 변환
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='thingindex')

print(ratings_matrix.shape)
ratings_matrix

In [ ]:
# thingindex를 기준으로 데이터프레임 병합
rating_thing = pd.merge(ratings, thing, on='thingindex')
rating_thing

In [ ]:
# columns='name' 로 name 컬럼으로 pivot 수행.
# 즉, 특산물 이름 컬럼으로 수행 
ratings_matrix = rating_thing.pivot_table('rating', index='userId', columns='name')
ratings_matrix

In [ ]:
# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

In [ ]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()  # 전치 행렬

In [ ]:
# 특산물과 특산물 간 코사인 유사도 산출
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 특산물명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

In [ ]:
# 포도와 유사한 특산물 6개 확인해보기
item_sim_df["포도"].sort_values(ascending=False)[:6]

# 자기 것 빼고 인셉션과 유사한 특산물 5개 확인해보기
item_sim_df["포도"].sort_values(ascending=False)[1:6]

In [ ]:
# 클릭 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 클릭 수를 계산하는 함수 정의
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [ ]:
# 클릭 수 예측
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred

In [ ]:
# 데이터프레임으로 변환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

In [ ]:
from sklearn.metrics import mean_squared_error

# 사용자 클릭수에 따른 특산물에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

In [ ]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 클릭수 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 클릭수 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 클릭수 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [ ]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 계산된 예측 클릭수 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

In [ ]:
# 사용자 9번에게 특산물을 추천해보자
# 추천에 앞서 9번 사용자가 높은 클릭수를 가진 특산물 확인해보면
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

In [ ]:
def get_unclick_things(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 특산물 정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 특산물을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 클릭한 특산물임. 대상 index를 추출하여 list 객체로 만듬
    already_click = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 특산물을 list 객체로 만듬. 
    things_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_click에 해당하는 thing는 things_list에서 제외함. 
    unclick_list = [ thing for thing in things_list if thing not in already_click]
    
    return unclick_list

In [ ]:
def recomm_thing_by_userid(pred_df, userId, unclick_list, top_n=10):
    # 예측 클릭 DataFrame에서 사용자id index와 unclick_list 들어온 특산물 컬럼을 추출하여
    # 가장 예측 클릭이 높은 순으로 정렬함. 
    recomm_thing = pred_df.loc[userId, unclick_list].sort_values(ascending=False)[:top_n]
    return recomm_thing

In [ ]:
# 사용자가 관람하지 않는 특산품명 추출   
unclick_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 특산품 추천 
recomm_thing = recomm_thing_by_userid(ratings_pred_matrix, 9, unclick_list, top_n=10)

# 클릭 수 데이타를 DataFrame으로 생성. 
recomm_thing = pd.DataFrame(data=recomm_thing.values,index=recomm_thing.index, columns=['pred_score'])
recomm_thing

# SVD - surprise 패키지

In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

from IPython.core.display import display, HTML
from pandas_profiling import ProfileReport
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [ ]:
# 원본 csv 파일은 hearder가 있는데, surprise를 적용하려면 header를 없애줘야 한다.
ratings = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/특산물 평점.xlsx')

In [ ]:
%%time
# ratings_noh.csv 파일로 저장하면 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/특산물 평점_noh.csv', index=False, header=False)

In [ ]:
%%time
from surprise import Reader
# reader로 파일 포멧 지정하기( 컬럼 명 4개 지정, 콤마로 구분, 평점 범위는 0.5 ~ 5점 )
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))

# 데이터 로딩하기
data=Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/특산물 평점_noh.csv',reader=reader)

In [ ]:
%%time
# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [ ]:
%%time
# SVD 알고리즘 적용(잠재 요인의 차원 수를 50개, 수행시마다 동일한 결과 도출을 위해 random_state 설정)
algo = SVD(n_factors=50, random_state=0)

In [ ]:
%%time 
# 학습셋으로 학습
algo.fit(trainset)

%%time
# 테스트셋으로 예상 평점 예측
predictions = algo.test(testset)

In [ ]:
%%time
# RMSE 평가
accuracy.rmse(predictions)

In [ ]:
# 교차 검증
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# GridSearchCV 세팅 : CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs

In [ ]:
%%time
# GridSearchCV로 학습
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
# 학습, 테스트 나누지않고 전체 학습

%%time
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/Colab Notebooks/특산물 평점_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()
# 실행시간 3분

In [ ]:
%%time
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)

algo.fit(trainset)

In [ ]:
# 특산물 대한 상세 속성 정보 DataFrame로딩
things = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/특산물.xlsx')
things

In [ ]:
# userId=9 의 특산물 데이터 추출하여 thingindex=42 데이터가 있는지 확인. 
thingIds = ratings[ratings['userId']==9]['thingindex']
if thingIds[thingIds==42].count() == 0:
    print('사용자 아이디 9는 특산물 아이디 42의 평점 없음')

print(things[things['thingindex']==42])

In [ ]:
# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
# 위와 같이 특정 사용자가 아직 안 본 전체 특산물 추출 후 예측 클릭 순으로 특산물 추천

def get_unseen_surprise(ratings, things, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 특산물을 리스트로 생성
    seen_thing = ratings[ratings['userId']== userId]['thingindex'].tolist()
    
    # 모든 특산물들의 thingindex를 리스트로 생성. 
    total_thing = things['thingindex'].tolist()
    
    # 모든 특산품들의 movieId중 이미 평점을 매긴 특산품의 movieId를 제외하여 리스트로 생성
    unseen_thing= [movie for movie in total_thing if movie not in seen_thing]
    print('평점 매긴 특산품 수:',len(seen_thing), '추천대상 특산품수:',len(unseen_thing), \
          '전체 특산품수:',len(total_thing))
    
    return unseen_thing

unseen_thing = get_unseen_surprise(ratings, things, 9)

In [ ]:
def recomm_thing_by_surprise(algo, userId, unseen_thing, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 클릭이 없는 특산물에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(thingindex)) for thingindex in unseen_thing]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 특산물의 정보 추출. 특산물 아이디, 추천 예상 클릭 수 , 특산물명 추출
    top_thing_ids = [ int(pred.iid) for pred in top_predictions]
    top_thing_rating = [ pred.est for pred in top_predictions]
    top_thing_name = things[things.thingindex.isin(top_thing_ids)]['name']
    top_thing_preds = [ (id, name, rating) for id, name, rating in zip(top_thing_ids, top_thing_name, top_thing_rating)]
    
    return top_thing_preds

In [ ]:
unseen_thing = get_unseen_surprise(ratings, things, 9)
top_thing_preds = recomm_thing_by_surprise(algo, 9, unseen_thing, top_n=10)
print("")
print('##### Top-10 추천 특산물 리스트 #####')

for top_thing in top_thing_preds:
    print(top_thing[1], ":", top_thing[2])